In [1]:
import numpy as np
import logging
from gen_inst import TSPInstance, load_dataset
from gls import guided_local_search
import time
import elkai
from tqdm import tqdm

In [2]:
perturbation_moves_map = {
    20: 5,
    50: 30,
    100: 40,
    200: 40,
}
iter_limit_map = {
    20: 73,
    50: 175,
    100: 1800,
    200: 800,
}
SCALE = 1000000
test_sizes = list(iter_limit_map.keys())

In [3]:
def heuristics_reevo(distance_matrix: np.ndarray) -> np.ndarray:
    # Calculate the average distance for each node
    average_distance = np.mean(distance_matrix, axis=1)

    # Calculate the distance ranking for each node
    distance_ranking = np.argsort(distance_matrix, axis=1)
    
    # Calculate the mean of the closest distances for each node
    closest_mean_distance = np.mean(distance_matrix[np.arange(distance_matrix.shape[0])[:, None], distance_ranking[:, 1:5]], axis=1)

    # Initialize the indicator matrix and calculate ratio of distance to average distance
    indicators = distance_matrix / average_distance[:, np.newaxis]

    # Set diagonal elements to np.inf
    np.fill_diagonal(indicators, np.inf)

    # Adjust the indicator matrix using the statistical measure
    indicators += closest_mean_distance[:, np.newaxis] / np.sum(distance_matrix, axis=1)[:, np.newaxis]

    return indicators

def vanilla_ktsp(distance_matrix: np.ndarray) -> np.ndarray:
    return distance_matrix

In [4]:
def calculate_cost(inst: TSPInstance, path: np.ndarray):
    return inst.distmat[path, np.roll(path, 1)].sum().item()

In [5]:
# Precompute the optimal solutions
optimal_objs_dict = {}
for problem_size in test_sizes:
    dataset_path = f"dataset/test{problem_size}_dataset.npy"
    dataset = load_dataset(dataset_path)
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path} with LKH")

    optimal_objs = []
    for i, instance in enumerate(tqdm(dataset, desc=f"tsp{problem_size}")):
        elkai_dist = elkai.DistanceMatrix(((instance.distmat * SCALE).astype(int)).tolist()) 
        optimal_route = elkai_dist.solve_tsp() # e.g. [0, 2, 1, 0]; with proven optimal solutions up to N=315 (https://github.com/fikisipi/elkai)
        optimal_obj = calculate_cost(instance, np.array(optimal_route[:-1]))
        optimal_objs.append(optimal_obj)
        
    mean_optimal_obj = np.mean(optimal_objs)

    optimal_objs_dict[problem_size] = mean_optimal_obj
    
print(optimal_objs_dict)

tsp200: 100%|██████████| 64/64 [02:13<00:00,  2.08s/it]

{20: 3.8362853943492015, 50: 5.68457994395107, 100: 7.778580370400294, 200: 10.71194600194464}


In [6]:
optimal_objs_dict = {20: 3.8362853943492015, 50: 5.68457994395107, 100: 7.778580370400294, 200: 10.71194600194464}

def solve(inst: TSPInstance, heuristics):
    start_time = time.time()
    heu = heuristics(inst.distmat.copy())
    result = guided_local_search(inst.distmat, heu, perturbation_moves_map[inst.n], iter_limit_map[inst.n])
    duration = time.time() - start_time
    return calculate_cost(inst, result), duration

def evaluate(function):
    print("[*] Function:", function.__name__, "\n")
    for problem_size in iter_limit_map.keys():
        dataset_path = f"dataset/test{problem_size}_dataset.npy"
        dataset = load_dataset(dataset_path)
        logging.info(f"[*] Evaluating {dataset_path}")

        objs = []
        durations = []
        for instance in tqdm(dataset):
            obj, duration = solve(instance, function)
            objs.append(obj)
            durations.append(duration)

        mean_obj = np.mean(objs).item()
        mean_optimal_obj = optimal_objs_dict[problem_size]
        gap = mean_obj / mean_optimal_obj - 1
        print(f"[*] Average for {problem_size}: {mean_obj:.6f} ({mean_optimal_obj:.6f})")
        print(f"[*] Optimality gap: {gap*100:.6f}%")
        print(f"[*] Total/Average duration: {sum(durations):.6f}s {sum(durations)/len(durations):.6f}s")
        print()

In [7]:
evaluate(heuristics_reevo)

[*] Function: heuristics_reevo 



100%|██████████| 64/64 [00:00<00:00, 939.19it/s]


[*] Average for 20: 3.836285 (3.836285)
[*] Optimality gap: 0.000000%
[*] Total/Average duration: 0.066535s 0.001040s



100%|██████████| 64/64 [00:02<00:00, 31.33it/s]


[*] Average for 50: 5.684580 (5.684580)
[*] Optimality gap: 0.000000%
[*] Total/Average duration: 2.030284s 0.031723s



100%|██████████| 64/64 [01:38<00:00,  1.54s/it]


[*] Average for 100: 7.778580 (7.778580)
[*] Optimality gap: 0.000000%
[*] Total/Average duration: 98.809937s 1.543905s



100%|██████████| 64/64 [02:41<00:00,  2.53s/it]

[*] Average for 200: 10.735125 (10.711946)
[*] Optimality gap: 0.216382%
[*] Total/Average duration: 161.792599s 2.528009s



In [8]:
evaluate(vanilla_ktsp)

[*] Function: vanilla_ktsp 



100%|██████████| 64/64 [00:00<00:00, 899.50it/s]


[*] Average for 20: 3.836448 (3.836285)
[*] Optimality gap: 0.004242%
[*] Total/Average duration: 0.069592s 0.001087s



  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [00:02<00:00, 31.38it/s]


[*] Average for 50: 5.685538 (5.684580)
[*] Optimality gap: 0.016862%
[*] Total/Average duration: 2.026035s 0.031657s



100%|██████████| 64/64 [01:39<00:00,  1.55s/it]


[*] Average for 100: 7.778761 (7.778580)
[*] Optimality gap: 0.002327%
[*] Total/Average duration: 99.047483s 1.547617s



100%|██████████| 64/64 [02:39<00:00,  2.50s/it]

[*] Average for 200: 10.742373 (10.711946)
[*] Optimality gap: 0.284046%
[*] Total/Average duration: 159.731591s 2.495806s

